In [1]:
##Logan Lauton

##importing packages needed for scrape
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time

##code below grabbed from https://raihanafiandi.medium.com/scraping-basketball-reference-data-using-python-f321c3f2903e
def single(season):
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_totals.html'
    table_html = BeautifulSoup(urlopen(url), 'html.parser').findAll('table')
    df = pd.read_html(str(table_html))[0]
    df = df.drop('Rk', 1) # drop Rk columns
    df.Player = df.Player.str.replace('*','') # remove asterisk on player's name
    df.insert(0,'Season',season) # insert season column
    df = df.apply(pd.to_numeric, errors='coerce').fillna(df) # convert non string values to numeric
    return df

##function taken from same link above, altered to sleep to avoid too many requests error 
def multiple(start_year,end_year):
    df = single(start_year)
    while start_year < end_year:
        time.sleep(4)                 ##code sleeps for 4 seconds between calls as 20 requests per minute
        start_year = start_year + 1   ##are allowed meaning only 15 requests per minute will be made here
        df = df.append(single(start_year))
    return df

In [2]:
##calling for all NBA seasons data except current(1949-50 until 2021-22) as current season is not yet finished
df = multiple(1950,2022)

/home/logan/py37/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  from ipykernel import kernelapp as app
/home/logan/py37/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  app.launch_new_instance()


In [3]:
##exporting df as csv
df.to_csv('NBA Player Stats(1950 - 2022).csv', index = True)